<a href="https://colab.research.google.com/github/chielgroen1998/RAAM/blob/main/RAAM_(RSI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install plotly


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import time

# Define the ticker symbols for the stocks
ticker_symbols = [
    "AAPL", "MSFT", "GOOG", "AMZN", "NVDA", "META", "TSLA", "PEP", "AVGO",
    "COST", "CSCO", "ADBE", "NFLX", "TMUS", "TXN", "CMCSA", "QCOM", "INTC", "HON",
    "AMD", "AMGN", "INTU", "ISRG", "BKNG", "MDLZ", "ADI", "LRCX", "VRTX", "MU",
    "AMAT", "SBUX", "GILD", "MRNA", "ADP", "PANW", "FISV", "CSX", "REGN", "MNST",
    "KLAC", "MAR", "NXPI", "ORLY", "ADSK", "MCHP", "AEP", "KDP", "SNPS",
    "FTNT", "IDXX", "LULU", "EXC", "CTAS", "PAYX", "XEL", "PCAR", "ODFL", "VRSK",
    "WBA", "CDNS", "AZN", "DLTR", "EBAY", "BIIB", "ROST", "CRWD", "CHTR",
    "FAST", "PDD", "ANSS", "MRVL", "TEAM", "WDAY", "BKR", "DDOG", "ZS", "CEG",
    "KHC", "VRSN", "CTSH", "SWKS", "OKTA", "EA", "LCID", "BIDU", "ALGN",
    "MELI", "JD", "LI", "NTES", "ASML", "DXCM", "CPRT"
]

# Parameters - all UTC timestamps
startdate = '2010-01-01'
enddate = '2025-12-31'
cutoff_date = '2014-07-01'

mom_p = 20 #26
vol_p = 35
RSI_p = 40
ass_amount = 6
MA_p = 40
cor_p = 40 # monthly

# Create reference timestamps with timezone
START_TS = pd.Timestamp(startdate).tz_localize('UTC')
END_TS = pd.Timestamp(enddate).tz_localize('UTC')
CUTOFF_TS = pd.Timestamp(cutoff_date).tz_localize('UTC')

def download_stock_data(ticker):
    """
    Download stock data for a single ticker with improved error handling.
    """
    try:
        print(f"\nDownloading data for {ticker}...")

        # Create a Ticker object
        stock = yf.Ticker(ticker)

        # Download the historical data
        data = stock.history(
            start=startdate,
            end=enddate,
            interval='1wk',
            auto_adjust=True  # This ensures we get adjusted prices
        )

        if data.empty:
            print(f"No data available for {ticker}")
            return None

        # Extract the closing prices
        prices = data['Close']  # Use 'Close' instead of 'Adj Close' since auto_adjust=True

        # Verify we have actual price data
        if len(prices) == 0:
            print(f"No price data for {ticker}")
            return None

        print(f"{ticker}: Got {len(prices)} prices from {prices.index[0]} to {prices.index[-1]}")

        # Add a small delay to avoid rate limiting
        time.sleep(1)  # Increased delay to be more conservative

        return prices

    except Exception as e:
        print(f"Error downloading {ticker}: {str(e)}")
        return None

def main():
    print("Starting download of stock data...")

    # Download and store the data
    all_data = {}
    successful_downloads = 0
    failed_downloads = 0
    long_history_tickers = []

    # Convert cutoff date to timezone-aware pandas timestamp
    cutoff = pd.Timestamp(cutoff_date, tz='UTC')

    # First pass: Download all data and identify stocks with sufficient history
    for ticker in ticker_symbols:
        series = download_stock_data(ticker)
        if series is not None and not series.empty:
            # Ensure index is timezone aware
            if series.index.tz is None:
                series.index = series.index.tz_localize('UTC')
            # Check if the stock has data from before our cutoff date
            if series.index[0] <= CUTOFF_TS:
                long_history_tickers.append(ticker)
                all_data[ticker] = series
                successful_downloads += 1
                print(f"Successfully processed {ticker} (full history)")
            else:
                print(f"Skipping {ticker} - insufficient history (starts from {series.index[0]})")
                failed_downloads += 1
        else:
            failed_downloads += 1

    # Create DataFrame and save results
    if all_data:
        # Convert to DataFrame
        combined_data = pd.DataFrame(all_data)

        # Save to CSV
        combined_data.to_csv('combined_stock_data.csv')

        print("\nDownload Summary:")
        print(f"Successfully downloaded: {successful_downloads} stocks")
        print(f"Failed downloads: {failed_downloads} stocks")
        print(f"Stocks with complete history from 2014: {len(long_history_tickers)}")
        print(f"\nShape of combined data: {combined_data.shape}")
        print("\nDate range in data:")
        print(f"Start: {combined_data.index[0]}")
        print(f"End: {combined_data.index[-1]}")
        print(f"\nStocks in dataset: {len(combined_data.columns)}")
        print("\nFirst few rows of the data:")
        print(combined_data.head())

        # Print list of included stocks
        print("\nIncluded stocks with complete history:")
        print(', '.join(sorted(long_history_tickers)))

    else:
        print("\nNo data was successfully downloaded!")
        print(f"Attempted downloads: {len(ticker_symbols)}")
        print(f"Failed downloads: {failed_downloads}")

if __name__ == "__main__":
    main()

Starting download of stock data...

AAPL: Got 814 prices from 2010-01-01 00:00:00-05:00 to 2025-08-01 00:00:00-04:00
Successfully processed AAPL (full history)

MSFT: Got 814 prices from 2010-01-01 00:00:00-05:00 to 2025-08-01 00:00:00-04:00
Successfully processed MSFT (full history)

GOOG: Got 814 prices from 2010-01-01 00:00:00-05:00 to 2025-08-01 00:00:00-04:00
Successfully processed GOOG (full history)

AMZN: Got 814 prices from 2010-01-01 00:00:00-05:00 to 2025-08-01 00:00:00-04:00
Successfully processed AMZN (full history)

NVDA: Got 814 prices from 2010-01-01 00:00:00-05:00 to 2025-08-01 00:00:00-04:00
Successfully processed NVDA (full history)

META: Got 691 prices from 2012-05-14 00:00:00-04:00 to 2025-08-04 00:00:00-04:00
Successfully processed META (full history)

TSLA: Got 789 prices from 2010-06-28 00:00:00-04:00 to 2025-08-04 00:00:00-04:00
Successfully processed TSLA (full history)

PEP: Got 814 prices from 2010-01-01 00:00:00-05:00 to 2025-08-01 00:00:00-04:00
Successfu

ERROR:yfinance:$FISV: possibly delisted; no timezone found


No data available for FISV

CSX: Got 814 prices from 2010-01-01 00:00:00-05:00 to 2025-08-01 00:00:00-04:00
Successfully processed CSX (full history)

REGN: Got 814 prices from 2010-01-01 00:00:00-05:00 to 2025-08-01 00:00:00-04:00
Successfully processed REGN (full history)

MNST: Got 814 prices from 2010-01-01 00:00:00-05:00 to 2025-08-01 00:00:00-04:00
Successfully processed MNST (full history)

KLAC: Got 814 prices from 2010-01-01 00:00:00-05:00 to 2025-08-01 00:00:00-04:00
Successfully processed KLAC (full history)

MAR: Got 814 prices from 2010-01-01 00:00:00-05:00 to 2025-08-01 00:00:00-04:00
Successfully processed MAR (full history)

NXPI: Got 784 prices from 2010-08-02 00:00:00-04:00 to 2025-08-04 00:00:00-04:00
Successfully processed NXPI (full history)

ORLY: Got 814 prices from 2010-01-01 00:00:00-05:00 to 2025-08-01 00:00:00-04:00
Successfully processed ORLY (full history)

ADSK: Got 814 prices from 2010-01-01 00:00:00-05:00 to 2025-08-01 00:00:00-04:00
Successfully process

In [3]:
def load_stock_data(filepath='combined_stock_data.csv'):
    combined_data = pd.read_csv(filepath, index_col=0, parse_dates=True)
    combined_data.index = pd.to_datetime(combined_data.index, utc=True)
    return combined_data

combined_data = load_stock_data()

In [15]:
price_changes = combined_data.pct_change(fill_method=None)
volatility = price_changes.rolling(window= vol_p ).std()
volatility_monthly = volatility.resample('ME').last()
ranked_volatility = volatility_monthly.rank(axis=1, method='first')

ranked_volatility

,AAPL,MSFT,GOOG,AMZN,NVDA,META,TSLA,PEP,AVGO,COST,...,SWKS,EA,BIDU,ALGN,MELI,JD,NTES,ASML,DXCM,CPRT
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-28 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
price_changes = combined_data.pct_change(fill_method=None)

mask1 = combined_data < combined_data.shift(mom_p)

moving_average = combined_data.rolling(window=MA_p).mean()
mask2 = combined_data < moving_average

price_changes = price_changes.where(~(mask1 | mask2))

momentum = price_changes.rolling(window=mom_p).apply(lambda x: (x + 1).prod() - 1)

momentum_monthly = momentum.resample('ME').last()

ranked_momentum = momentum_monthly.rank(axis=1, method='first', ascending=False)

ranked_momentum.to_csv('ranked_momentum.csv')

ranked_momentum

,AAPL,MSFT,GOOG,AMZN,NVDA,META,TSLA,PEP,AVGO,COST,...,SWKS,EA,BIDU,ALGN,MELI,JD,NTES,ASML,DXCM,CPRT
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-28 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
pct_change_df = combined_data.pct_change(fill_method=None)

correlation_means = []

for index, row in pct_change_df.iterrows():
    other_tickers = [ticker for ticker in pct_change_df.columns if ticker != index]
    correlation_mean = row.corr(pct_change_df[other_tickers].mean(axis=1))
    correlation_means.append(correlation_mean)

pct_change_df['Correlation_Mean'] = correlation_means
pct_change_df = pct_change_df.drop('Correlation_Mean', axis=1)

resampled_df = pct_change_df.resample('ME').mean()
rolling_mean_df = resampled_df.rolling(window=cor_p).mean()
rolling_mean_df = rolling_mean_df.shift(-1)
rolling_mean_df = rolling_mean_df.iloc[:-1]

rankings_df = rolling_mean_df

ranked_correlation = rolling_mean_df.rank(axis=1, method='first')

ranked_correlation

,AAPL,MSFT,GOOG,AMZN,NVDA,META,TSLA,PEP,AVGO,COST,...,SWKS,EA,BIDU,ALGN,MELI,JD,NTES,ASML,DXCM,CPRT
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-28 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-04-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
combined_data.index = pd.to_datetime(combined_data.index)

rsi_values = pd.DataFrame(index=combined_data.index)

for stock in combined_data.columns:
    stock_changes = combined_data[stock].pct_change(fill_method=None)

    gain = stock_changes.clip(lower=0)
    loss = -stock_changes.clip(upper=0)

    avg_gain = gain.ewm(com=RSI_p, adjust=False).mean()
    avg_loss = loss.ewm(com=RSI_p, adjust=False).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    rsi_values[stock] = rsi

rsi_monthly = rsi_values.resample('ME').last()

ranked_rsi = rsi_monthly.rank(axis=1, method='first', ascending=False)

# Print the ranked RSI
ranked_rsi

,AAPL,MSFT,GOOG,AMZN,NVDA,META,TSLA,PEP,AVGO,COST,...,SWKS,EA,BIDU,ALGN,MELI,JD,NTES,ASML,DXCM,CPRT
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-28 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
ranking_weights = pd.Series({
    'Momentum Score': 0,
    'RSI Score': 1,
    'Volatility Score': 0,
    'Correlation Score': 0
})

weighted_momentum = ranked_momentum * ranking_weights['Momentum Score']
weighted_rsi = ranked_rsi * ranking_weights['RSI Score']
weighted_volatility = ranked_volatility * ranking_weights['Volatility Score']
weighted_correlation = ranked_correlation * ranking_weights['Correlation Score']

cumulative_score = weighted_momentum + weighted_rsi + weighted_volatility + weighted_correlation

cumulative_score.to_csv('cumscore.csv', index=True)

cumulative_score

,AAPL,MSFT,GOOG,AMZN,NVDA,META,TSLA,PEP,AVGO,COST,...,SWKS,EA,BIDU,ALGN,MELI,JD,NTES,ASML,DXCM,CPRT
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-28 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-05-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-06-30 00:00:00+00:00,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,16.0,NaN,5.0,NaN,NaN,NaN


In [9]:
import pandas as pd

def process_portfolio_selections(cumulative_score, ass_amount):
    ranked_df = cumulative_score.apply(lambda x: x.nsmallest(ass_amount), axis=1)

    ranked_mask = ranked_df.notna()

    result_df = pd.DataFrame(index=ranked_df.index, columns=ranked_df.columns)
    for column in ranked_df.columns:
        result_df[column] = ranked_mask[column].map({True: column, False: 0})
    result_df.index = pd.to_datetime(result_df.index)
    result_df.index = result_df.index + pd.DateOffset(months=1)

    dates = []
    tickers = []

    for date, row in result_df.iterrows():
        valid_tickers = row[row != 0]
        if not valid_tickers.empty:
            dates.extend([date] * len(valid_tickers))
            tickers.extend(valid_tickers.index)

    non_zero_df = pd.DataFrame({
        'Date': dates,
        'Ticker': tickers
    })

    non_zero_df['Date'] = pd.to_datetime(non_zero_df['Date'])

    return non_zero_df

portfolio_selections = process_portfolio_selections(cumulative_score, ass_amount)

portfolio_selections.to_csv('portfolio_selections.csv', index=False)

print("\nPortfolio Selections Summary:")
print(f"Total number of selections: {len(portfolio_selections)}")
print(f"Date range: {portfolio_selections['Date'].min()} to {portfolio_selections['Date'].max()}")
print(f"Number of unique tickers: {portfolio_selections['Ticker'].nunique()}")
print("\nFirst few selections:")
print(portfolio_selections.head(50))
print(portfolio_selections.tail(6))


Portfolio Selections Summary:
Total number of selections: 876
Date range: 2013-04-30 00:00:00+00:00 to 2025-08-31 00:00:00+00:00
Number of unique tickers: 80

First few selections:
                        Date Ticker
0  2013-04-30 00:00:00+00:00   ADBE
1  2013-04-30 00:00:00+00:00   BIIB
2  2013-04-30 00:00:00+00:00   CHTR
3  2013-04-30 00:00:00+00:00   GILD
4  2013-04-30 00:00:00+00:00    PEP
5  2013-04-30 00:00:00+00:00    WBA
6  2013-05-30 00:00:00+00:00   ADBE
7  2013-05-30 00:00:00+00:00    AEP
8  2013-05-30 00:00:00+00:00   BIIB
9  2013-05-30 00:00:00+00:00   GILD
10 2013-05-30 00:00:00+00:00   NFLX
11 2013-05-30 00:00:00+00:00    PEP
12 2013-06-30 00:00:00+00:00   BIIB
13 2013-06-30 00:00:00+00:00   CHTR
14 2013-06-30 00:00:00+00:00   DXCM
15 2013-06-30 00:00:00+00:00   MSFT
16 2013-06-30 00:00:00+00:00     MU
17 2013-06-30 00:00:00+00:00   REGN
18 2013-07-30 00:00:00+00:00   ADBE
19 2013-07-30 00:00:00+00:00   CHTR
20 2013-07-30 00:00:00+00:00   DXCM
21 2013-07-30 00:00:00+00:

In [10]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import numpy as np
from typing import Dict, List, Any

def calculate_stock_return(ticker: str, start_date: pd.Timestamp, end_date: pd.Timestamp) -> Dict[str, Any]:
    """Calculate return for a single stock with error handling"""
    try:
        stock = yf.download(
            ticker,
            start=start_date,
            end=end_date + timedelta(days=1),
            progress=False,
            ignore_tz=True,
            auto_adjust=False # Explicitly set auto_adjust to False
        )

        if stock.empty or len(stock) < 2:
            print(f"Warning: Insufficient data for {ticker} between {start_date} and {end_date}")
            return None

        # Use 'Adj Close' when auto_adjust is False
        first_price = float(stock['Adj Close'].iloc[0].item())
        last_price = float(stock['Adj Close'].iloc[-1].item())
        pct_change = ((last_price - first_price) / first_price) * 100

        return {
            'Start_Price': first_price,
            'End_Price': last_price,
            'Return_Pct': pct_change
        }
    except Exception as e:
        print(f"Error processing {ticker} for period {start_date} to {end_date}: {str(e)}")
        return None

def analyze_portfolio(df: pd.DataFrame) -> pd.DataFrame:
    """Analyze portfolio returns"""
    results = []

    # Group by date to get monthly portfolios
    monthly_portfolios = df.groupby('Date')['Ticker'].apply(list).reset_index()

    total_tickers = sum(len(tickers) for tickers in monthly_portfolios['Ticker'])
    processed = 0

    for _, row in monthly_portfolios.iterrows():
        date = pd.to_datetime(row['Date'])
        tickers = row['Ticker']

        # Calculate start and end of month
        start_date = date.replace(day=1)
        end_date = (start_date + pd.offsets.MonthEnd(0))

        for ticker in tickers:
            return_data = calculate_stock_return(ticker, start_date, end_date)
            processed += 1

            if return_data is not None:
                results.append({
                    'Date': date,
                    'Ticker': ticker,
                    **return_data
                })

            # Print progress
            if processed % 100 == 0:
                print(f"Processed {processed}/{total_tickers} stocks")

    return pd.DataFrame(results)

def format_summary(summary_df: pd.DataFrame) -> pd.DataFrame:
    """Format the summary dataframe for better display"""
    summary_df.index = summary_df.index.strftime('%Y-%m-%d')
    return summary_df

def print_analysis(returns_df: pd.DataFrame, summary_df: pd.DataFrame):
    """Print formatted analysis results"""
    print("\nPortfolio Analysis Summary:")
    print(f"Total periods analyzed: {len(summary_df)}")
    print(f"Total stocks analyzed: {len(returns_df)}")

    print("\nFirst few rows of monthly summary:")
    print(format_summary(summary_df.head()))

    print("\nOverall Statistics:")
    print(f"Average monthly return: {returns_df['Return_Pct'].mean():.2f}%")
    print(f"Best monthly return: {returns_df['Return_Pct'].max():.2f}%")
    print(f"Worst monthly return: {returns_df['Return_Pct'].min():.2f}%")
    print(f"Return standard deviation: {returns_df['Return_Pct'].std():.2f}%")

    # Calculate annualized statistics
    monthly_returns = returns_df.groupby('Date')['Return_Pct'].mean()
    annualized_return = ((1 + monthly_returns/100).prod() ** (12/len(monthly_returns)) - 1) * 100
    annualized_vol = monthly_returns.std() * np.sqrt(12)

    print(f"\nAnnualized Statistics:")
    print(f"Annualized Return: {annualized_return:.2f}%")
    print(f"Annualized Volatility: {annualized_vol:.2f}%")
    print(f"Sharpe Ratio (Rf=0): {(annualized_return/annualized_vol):.2f}")

# Example usage:
if __name__ == "__main__":
    # Read your CSV data
    portfolio_df = pd.read_csv("portfolio_selections.csv")

    # Process returns
    print("Starting portfolio analysis...")
    returns_df = analyze_portfolio(portfolio_df)

    # Generate summary
    summary_df = returns_df.groupby('Date').agg({
        'Return_Pct': [
            ('Mean Return %', 'mean'),
            ('Std Dev %', 'std'),
            ('Min Return %', 'min'),
            ('Max Return %', 'max'),
            ('Count', 'count')
        ]
    }).round(2)

    # Flatten column names
    summary_df.columns = summary_df.columns.get_level_values(1)

    # Save results
    returns_df.to_csv('stock_returns_detailed.csv', index=False)
    summary_df.to_csv('monthly_summary.csv')

    # Print analysis
    print_analysis(returns_df, summary_df)

Starting portfolio analysis...
Processed 100/876 stocks
Processed 200/876 stocks
Processed 300/876 stocks
Processed 400/876 stocks
Processed 500/876 stocks
Processed 600/876 stocks
Processed 700/876 stocks
Processed 800/876 stocks

Portfolio Analysis Summary:
Total periods analyzed: 149
Total stocks analyzed: 876

First few rows of monthly summary:
            Mean Return %  Std Dev %  Min Return %  Max Return %  Count
Date                                                                   
2013-04-30           4.96       5.26         -2.93         13.57      6
2013-05-30           1.71       8.08         -9.69         10.81      6
2013-06-30           4.08      10.87         -5.36         22.17      6
2013-07-30           1.66       6.47         -7.22          9.01      6
2013-08-31          -3.74       2.12         -5.43          0.41      6

Overall Statistics:
Average monthly return: 2.02%
Best monthly return: 67.78%
Worst monthly return: -42.04%
Return standard deviation: 8.91%

An

In [31]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def calculate_monthly_returns_for_ticker(ticker: str, start_date: pd.Timestamp, end_date: pd.Timestamp) -> pd.Series:
    """Download monthly adjusted close prices and calculate monthly percentage change."""
    try:
        data = yf.download(
            ticker,
            start=start_date,
            end=end_date + timedelta(days=1), # Add a day to ensure end date is included
            interval='1mo', # Monthly interval
            progress=False,
            ignore_tz=True,
            auto_adjust=True # Use auto_adjust for adjusted prices
        )

        if data.empty:
            print(f"Warning: No data available for {ticker} between {start_date} and {end_date}")
            return pd.Series([], dtype='float64')

        # Calculate monthly percentage change from adjusted close prices
        monthly_returns = data['Close'].pct_change().dropna()

        return monthly_returns * 100 # Return in percentage

    except Exception as e:
        print(f"Error downloading data for {ticker}: {str(e)}")
        return pd.Series([], dtype='float64')

def calculate_and_print_statistics(monthly_returns: pd.Series, ticker: str):
    """Calculate and print portfolio statistics for a given ticker."""
    if monthly_returns.empty:
        print(f"\nNo data to calculate statistics for {ticker}.")
        return

    print(f"\n{ticker} Statistics:")

    # Overall Statistics
    print("\nOverall Statistics:")
    print(f"Average monthly return: {monthly_returns.mean().item():.2f}%")
    print(f"Best monthly return: {monthly_returns.max().item():.2f}%")
    print(f"Worst monthly return: {monthly_returns.min().item():.2f}%")
    print(f"Return standard deviation: {monthly_returns.std().item():.2f}%")

    # Annualized Statistics
    # Ensure there are enough periods for annualization
    if len(monthly_returns) > 0:
        total_return = ((1 + monthly_returns/100).prod() - 1) * 100
        annualized_return = ((1 + monthly_returns/100).prod() ** (12/len(monthly_returns)) - 1) * 100
        annualized_vol = monthly_returns.std() * np.sqrt(12)
        sharpe_ratio = (annualized_return - 0.02) / annualized_vol # Using 2% risk-free rate

        print("\nAnnualized Statistics:")
        print(f"Total Return: {total_return.item():.2f}%")
        print(f"Annualized Return: {annualized_return.item():.2f}%")
        print(f"Annualized Volatility: {annualized_vol.item():.2f}%")
        print(f"Sharpe Ratio (Rf=2%): {sharpe_ratio.item():.2f}")

        # Max Drawdown
        cumulative_returns = (1 + monthly_returns/100).cumprod()
        max_drawdown = (cumulative_returns / cumulative_returns.cummax() - 1).min() * 100
        print(f"Max Drawdown: {max_drawdown.item():.2f}%")
    else:
        print("\nInsufficient data to calculate annualized statistics.")


# Define the date range based on your portfolio data
# Assuming portfolio_selections is available and has 'Date' column
if 'portfolio_selections' in locals() and not portfolio_selections.empty:
    start_date = portfolio_selections['Date'].min()
    end_date = portfolio_selections['Date'].max()
else:
    # Fallback dates if portfolio_selections is not available
    start_date = pd.Timestamp('2013-04-30', tz='UTC')
    end_date = pd.Timestamp('2025-08-31', tz='UTC')


# Get monthly returns for SPY and QQQ
spy_monthly_returns = calculate_monthly_returns_for_ticker('SPY', start_date, end_date)
qqq_monthly_returns = calculate_monthly_returns_for_ticker('QQQ', start_date, end_date)

# Calculate and print statistics for SPY and QQQ
calculate_and_print_statistics(spy_monthly_returns, 'SPY')
calculate_and_print_statistics(qqq_monthly_returns, 'QQQ')


SPY Statistics:

Overall Statistics:
Average monthly return: 1.16%
Best monthly return: 13.36%
Worst monthly return: -13.00%
Return standard deviation: 4.27%

Annualized Statistics:
Total Return: 376.37%
Annualized Return: 13.59%
Annualized Volatility: 14.79%
Sharpe Ratio (Rf=2%): 0.92
Max Drawdown: -23.97%

QQQ Statistics:

Overall Statistics:
Average monthly return: 1.60%
Best monthly return: 15.22%
Worst monthly return: -13.49%
Return standard deviation: 5.11%

Annualized Statistics:
Total Return: 749.24%
Annualized Return: 19.08%
Annualized Volatility: 17.72%
Sharpe Ratio (Rf=2%): 1.08
Max Drawdown: -32.65%


In [42]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import yfinance as yf # Import yfinance

# Assuming 'monthly_returns' is available from previous cells (portfolio monthly returns)

# Calculate Portfolio Statistics
portfolio_total_return = ((1 + monthly_returns).prod() - 1) * 100
portfolio_annual_return = ((1 + monthly_returns).prod() ** (12/len(monthly_returns)) - 1) * 100
portfolio_monthly_vol = monthly_returns.std() * 100
portfolio_annual_vol = monthly_returns.std() * np.sqrt(12) * 100
portfolio_sharpe_ratio = ((monthly_returns.mean() - 0.02/12) / (monthly_returns.std()) * np.sqrt(12))
portfolio_cum_returns = (1 + monthly_returns).cumprod()
portfolio_max_drawdown = ((portfolio_cum_returns / portfolio_cum_returns.cummax() - 1).min() * 100)


# Download and process benchmark data directly
benchmark_tickers = ['SPY', 'QQQ']
benchmark_monthly_returns = {}
start_date = monthly_returns.index.min()
end_date = monthly_returns.index.max()

for ticker in benchmark_tickers:
    print(f"\nDownloading monthly data for {ticker}...")
    benchmark_data = yf.download(
        ticker,
        start=start_date,
        end=end_date + pd.DateOffset(days=1), # Add a day to ensure end date is included
        interval='1mo', # Monthly interval
        progress=False,
        ignore_tz=True,
        auto_adjust=True # Use auto_adjust for adjusted prices
    )
    if not benchmark_data.empty:
        # Calculate monthly percentage change and convert to %
        monthly_returns_bm = benchmark_data['Close'].pct_change().dropna() * 100
        # Reindex to align with portfolio monthly returns and fill NaNs with 0
        benchmark_monthly_returns[ticker] = monthly_returns_bm.reindex(monthly_returns.index).fillna(0)
    else:
        print(f"No monthly data available for {ticker}")
        benchmark_monthly_returns[ticker] = pd.Series(0.0, index=monthly_returns.index) # Create a Series of zeros if no data


# Prepare data for the comparison table
comparison_data = {
    'Metric': ['Total Return (%)', 'Annual Return (%)', 'Monthly Volatility (%)', 'Annual Volatility (%)', 'Sharpe Ratio (Rf=2%)', 'Max Drawdown (%)'],
    'Portfolio': [portfolio_total_return, portfolio_annual_return, portfolio_monthly_vol, portfolio_annual_vol, portfolio_sharpe_ratio, portfolio_max_drawdown]
}

# Calculate and add benchmark statistics to the comparison data
for ticker, monthly_returns_bm in benchmark_monthly_returns.items():
    if not monthly_returns_bm.empty:
        # Calculate scalar statistics for benchmarks
        total_return = ((1 + monthly_returns_bm/100).prod() - 1) * 100
        annual_return = ((1 + monthly_returns_bm/100).prod() ** (12/len(monthly_returns_bm)) - 1) * 100
        monthly_vol = monthly_returns_bm.std() * 100
        annual_vol = monthly_returns_bm.std() * np.sqrt(12) * 100
        sharpe_ratio = ((monthly_returns_bm.mean() - 0.02/12) / (monthly_returns_bm.std()) * np.sqrt(12))
        cum_returns_bm = (1 + monthly_returns_bm/100).cumprod()
        max_drawdown = ((cum_returns_bm / cum_returns_bm.cummax() - 1).min()) * 100

        comparison_data[ticker] = [total_return, annual_return, monthly_vol, annual_vol, sharpe_ratio, max_drawdown]
    else:
        comparison_data[ticker] = [np.nan] * len(comparison_data['Metric'])


comparison_df = pd.DataFrame(comparison_data)
# Format the table for better readability
comparison_df = comparison_df.round(2)
display(comparison_df)

# Create Cumulative Gains Line Chart
fig = go.Figure()

# Add Portfolio cumulative returns
fig.add_trace(go.Scatter(x=portfolio_cum_returns.index, y=portfolio_cum_returns,
                         mode='lines',
                         name='Portfolio'))

# Add Benchmark cumulative returns
if 'benchmark_monthly_returns' in locals():
    for ticker, monthly_returns_bm in benchmark_monthly_returns.items():
        # Ensure monthly_returns_bm is a pandas Series and not empty before calculating cumulative returns
        if isinstance(monthly_returns_bm, pd.Series) and not monthly_returns_bm.empty:
            cum_returns_bm = (1 + monthly_returns_bm/100).cumprod() # Calculate cumulative returns from percentage
            fig.add_trace(go.Scatter(x=cum_returns_bm.index, y=cum_returns_bm,
                                     mode='lines',
                                     name=ticker))
        else:
            print(f"Skipping cumulative plot for {ticker}: Monthly returns data is not a valid Series or is empty.")


fig.update_layout(
    title='Cumulative Gains Comparison',
    xaxis_title='Date',
    yaxis_title='Cumulative Return',
    hovermode='x unified'
)

fig.show()

,Metric,Portfolio,SPY,QQQ
0,Total Return (%),1328.94,Ticker SPY 0.0 dtype: float64,Ticker QQQ 0.0 dtype: float64
1,Annual Return (%),23.89,Ticker SPY 0.0 dtype: float64,Ticker QQQ 0.0 dtype: float64
2,Monthly Volatility (%),5.83,Ticker SPY 0.0 dtype: float64,Ticker QQQ 0.0 dtype: float64
3,Annual Volatility (%),20.19,Ticker SPY 0.0 dtype: float64,Ticker QQQ 0.0 dtype: float64
4,Sharpe Ratio (Rf=2%),1.07,Ticker SPY -inf dtype: float64,Ticker QQQ -inf dtype: float64
5,Max Drawdown (%),-27.12,Ticker SPY 0.0 dtype: float64,Ticker QQQ 0.0 dtype: float64


Skipping cumulative plot for SPY: Monthly returns data is not a valid Series or is empty.
Skipping cumulative plot for QQQ: Monthly returns data is not a valid Series or is empty.
